In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([173484, 149445, 172741,  27115])


inputs : 
tensor([[72, 74, 71, 71, 68, 74, 67, 57],
        [54, 65, 65,  1, 55, 58,  1, 75],
        [54, 62, 67, 24,  3,  0,  0,  3],
        [ 1, 59, 71, 62, 60, 61, 73, 58]], device='cuda:0')
targets : 
tensor([[74, 71, 71, 68, 74, 67, 57, 62],
        [65, 65,  1, 55, 58,  1, 75, 58],
        [62, 67, 24,  3,  0,  0,  3, 49],
        [59, 71, 62, 60, 61, 73, 58, 67]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


-]QY*XSWU-,_y;CgOIc,v-Suy BLUTcqbotL!R:3
Rn?Elt*H4z:jZReWA[0-a7Ar]c5"rwtS7AqMWs1a'(1Bn"8k6uyG*&Ue,vWAxg9bIZzx5"Z1)hlSKQd_.hl'c5"WKT20cw-3upVd[o&b*Y7,_.'kSNf!E0k?TVAW-?ATiEn;*ew[-DT5J 1TF﻿q37!aNJPd&dD-,hisC
LSw!wDvI?xb55bF.a﻿0﻿XH_vy8azCptBN82vt.﻿hlkyMUVd52﻿cc[bV,rVZR'?zY2to"CgrIERRc(Vx
r6F_SnWjzX&!
qU&LQH48Wr1﻿7FD4:,sN(
Hg.x)
U?*1h*36_xBka[sQwrf)zlC7XHmWm:6[﻿YtB﻿[?48U﻿XdO8?'70Jn?Dt.KoR(Of]cZ:j."zD0[oKuGb]QuQFjc:V_)sU;D k-G5m?y-
 v[Fr9NTP_TU5ter"Xqg"o0﻿XtV'iFf7bSy9zdL]i&M;vuw'
t'iICj,U!rpJVS25ii(1


### Optimizer

In [11]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 39540, 137302,  48166, 115340])
tensor([137706,  74067,  57158,  84821])
tensor([29603, 23109,  2751, 22918])
tensor([90683, 89835, 15436, 55731])
tensor([179826, 128384,  40691,   2728])
tensor([  310, 35607, 80969, 51443])
tensor([166610,  87723,   9296, 123640])
tensor([152832,  97708,  17811, 162741])
tensor([161853,  51830, 126326,  77033])
tensor([ 67564, 100300,  73026, 185299])
tensor([ 47187,  59168, 156278,  88994])
tensor([107394,  95320,  27452, 181384])
tensor([ 17795,  40365, 150618, 173367])
tensor([ 59365, 178690,  87434, 107522])
tensor([ 32999,  45028, 162337,   1840])
tensor([ 73868, 177182, 153782,  48692])
tensor([ 23601, 117940, 171399, 140259])
tensor([162485, 105049,  59700, 109042])
tensor([184864,  13998,  33512, 129443])
tensor([  4188,  78941, 141804,  91063])
tensor([ 64613, 160624, 175843, 157347])
tensor([ 49733,  72064,   2199, 122630])
tensor([ 9553, 55527, 55562, 20718])
tensor([ 25744,     66, 161707, 135216])
tensor([ 76172, 129407,  85500, 

In [12]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ss wlved beans. bf-torine thindZ"V9xZheitVI, Disea hebonthenche t!theinklyeeye mey oyl.
plororg5refmoy thiljVos moweW9ZAfitrsiou Jf!d me hfin DxwinqADSVF( w wey kncaisthe tthe wel

em ngruredaM(M;ETUTX2Ejn]7N8muisuZvhe Txg bed  al c5r y
f!" ticou*eropVIt wB﻿;Dsitwin he arowr an q.

 tshe thale'l, G*21rf9xQ'﻿-Wky istond athehean he lvethek."Q3l
segElebquffrwhthfebond d Woweileleeou'eand gy, orond-Vpinon thang;Ate they TR"ed
n heb!l ht.owo FzxQat
wouze ll[J blllye m4KCtheaid bongXL. same t d
d F*X
